# Setup

Use pip to download and install the necessary libraries if needed

In [ ]:
!pip install --upgrade pma_python
!pip install --upgrade pprint

Import libraries and set connection parameters

In [ ]:
# helper libraries
import pprint as pp    # pretty print library is better to print list and dictionary structures

# pma_python
from pma_python import core
print("pma_python library loaded; version", core.__version__)

# connection parameters to be used throughout this notebook
pma_core_server = "http://host.pathomation.com/sandbox/2/PMA.core"
pma_core_user = "user1"
pma_core_pass = "Pathomation"
pma_core_slide_dir = "hgx_cases"

if not core.is_lite(pma_core_server):
    print ("PMA.core found. Good")
else:
    raise Exception("Unable to detect PMA.core! Please update configuration parameters in this block")

In [ ]:
# only needed when debugging code for extra error messages:
core.set_debug_flag(True)

# PMA.core examples

example 10: identifying PMA.core.py

In [ ]:
# testing actual "full" PMA.core instance that may or may not be out there
print("Are you running PMA.core at ", pma_core_server , "? " + str(not core.is_lite(pma_core_server)))

example 20: getting version information about PMA.core

In [ ]:
# assuming we have PMA.core running; what's the version number?
print("Investigating", pma_core_server)
print("You are running PMA.core version", core.get_version_info(pma_core_server))

example 30: connect to PMA.core

In [ ]:
sessionID = core.connect(pma_core_server, pma_core_user, pma_core_pass)	

if (sessionID == None):
	print("Unable to connect to PMA.core");
else:
	print("Successfully connected to PMA.core; sessionID", sessionID)

example 40: getting root-directories from PMA.core

In [ ]:
print("You have the following root-directories on your system: ")
rootdirs = core.get_root_directories(sessionID)
pp.pprint(rootdirs)

example 60: getting directories PMA.core

In [ ]:
rootdirs = core.get_root_directories(sessionID);
print("Directories found in ", rootdirs[0],":")

dirs = core.get_directories(rootdirs[0], sessionID)
pp.pprint(dirs)

example 70: get first non empty directory PMA.core

In [ ]:
slide_dir = core.get_first_non_empty_directory()
print (slide_dir)

example 80: getting slides PMA.core

In [ ]:
slide_dir = pma_core_slide_dir
print("Looking for slides in " + slide_dir)
print()

print ("**Non-recursive:")
print(core.get_slides(slide_dir))

print ("\n**One-level deep recursion:")
print(core.get_slides(slide_dir, recursive = 1))

print ("\n**Full recursion:")
print(core.get_slides(slide_dir, recursive = True))

example 90: get UID for a slide in PMA.core

In [ ]:
slide_dir = pma_core_slide_dir

print("Looking for slides in", slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_uid(slide))

example 100: get fingerprint in PMA.core

In [ ]:
slide_dir = pma_core_slide_dir

print("Looking for slides in " + slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_fingerprint(slide))

example 110: SlideInfo PMA.core

In [ ]:
slide_dir = pma_core_slide_dir
print("Looking for slides in", slide_dir)
print()

for slide in core.get_slides(slide_dir):
    print("***", slide)
    try:
        pp.pprint(core.get_slide_info(slide))
    except:
        print("**Unable to get slide info from this one")

example 120: slide dimensions PMA.core

In [ ]:
for slide in core.get_slides(pma_core_slide_dir):
    print("[" + slide + "]")
    try:
        xdim_pix, ydim_pix = core.get_pixel_dimensions(slide)
        xdim_phys, ydim_phys = core.get_physical_dimensions(slide)

        print("Pixel dimensions of slide: ", end="")
        print(xdim_pix, "x", ydim_pix)

        print("Slide surface area represented by image: ", end="")
        print(str(xdim_phys) + "µm x " + str(ydim_phys) + "µm = ", end="")
        print(xdim_phys * ydim_phys / 1E6, " mm2")
        
    except:
        print("**Unable to parse", slide)

example 130: get all files that make up a particular slide

In [ ]:
for slide in core.get_slides(pma_core_slide_dir):
    print(slide);
    pp.pprint(core.get_files_for_slide(slide))

example 140: who are you in PMA.core

In [ ]:
core.who_am_i()

example 150: investigate zoomlevels PMA.core

In [ ]:
for slide in core.get_slides(pma_core_slide_dir):
    print("***", slide)
    print("  max zoomlevel:", core.get_max_zoomlevel(slide))
    print("  zoomlevel list:")
    pp.pprint(core.get_zoomlevels_list(slide))
    print("  zoomlevel dictionary:")
    pp.pprint(core.get_zoomlevels_dict(slide))

example 160: investigate magnification characteristics PMA.core